<a href="https://colab.research.google.com/github/ShiyuDeng-Simon/MeddiConnect/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install transformers accelerate flask

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
model_name = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", use_auth_token=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [21]:
def generate_response(prompt, max_length=1000):
    """
    Generates a response using the Llama model.

    Args:
      prompt (str): The input prompt text.
      max_length (int): The maximum length of the generated response.

    Returns:
      str: The generated response text.
    """
    formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{prompt}\n<|start_header_id|>assistant<|end_header_id|>\n"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=max_length)
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [22]:
prompt = "What is the meaning of life?"
response = generate_response(prompt)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


user
What is the meaning of life?
assistant
The question of the meaning of life has puzzled philosophers, theologians, scientists, and everyday people for centuries. There is no one definitive answer, and different perspectives offer varying interpretations. Here are some of the most common approaches:

**Philosophical perspectives:**

1. **Existentialism**: Life has no inherent meaning; we create our own meaning through our choices and experiences.
2. **Absurdism**: Life is inherently meaningless, but we must create our own purpose and meaning in the face of absurdity.
3. **Utilitarianism**: The meaning of life is to maximize happiness and well-being for the greatest number of people.
4. **Stoicism**: The meaning of life is to live in accordance with reason, virtue, and self-control.
5. **Humanism**: The meaning of life is to cultivate human flourishing, happiness, and fulfillment.

**Religious perspectives:**

1. **Theism**: The meaning of life is to fulfill a divine plan or purpose,

In [25]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    """
    API endpoint to generate text based on provided prompt.
    Expects a JSON POST request with a 'prompt' key.
    Returns:
        JSON response with a 'response' key.
    """
    data = request.get_json()
    if not data or 'prompt' not in data:
        return jsonify({'error': 'Missing prompt in request body'}), 400
    prompt = data['prompt']
    try:
        response = generate_response(prompt)
        return jsonify({'response': response})
    except Exception as e:
        return jsonify({'error': str(e)}), 500


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
